In [7]:
from keras.models import load_model
import tensorflow as tf
import pandas as pd
import numpy as np
import librosa
import IPython.display as ipd
import csv
import os
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
song_data=pd.read_csv('song_data_reco.csv',index_col=0)
scaler = StandardScaler()

# Model Load

In [9]:
model =load_model('model_best3.h5')
model.summary()

Model: "sequential_139"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_556 (Dense)           (None, 120)               11640     
                                                                 
 dropout_412 (Dropout)       (None, 120)               0         
                                                                 
 dense_557 (Dense)           (None, 45)                5445      
                                                                 
 dropout_413 (Dropout)       (None, 45)                0         
                                                                 
 dense_558 (Dense)           (None, 30)                1380      
                                                                 
 dropout_414 (Dropout)       (None, 30)                0         
                                                                 
 dense_559 (Dense)           (None, 4)              

In [10]:
 genre_dict = {
    0: '랩/힙합', 
    1: '록/메탈',
    2: '발라드',
    3: '클래식',
    }

# song 파일 안의 wav파일을 csv로 변환

In [16]:
for songname in os.listdir('song'):
    y,sr=librosa.load(f'song/{songname}',sr=16000)
    mel = librosa.feature.melspectrogram(y=y,sr=sr,win_length=512,hop_length=256,n_mels=48)
    mel_db=librosa.power_to_db(mel,ref=np.max)
    header=''
    for i in range(1, 49):
        header += f' mel_mean{i}' 
        header += f' mel_var{i}'
    header = header.split()
    to_append=''

    file = open(f'{songname}.csv', 'w', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(header)
    for e in mel_db:
        to_append += f' {np.mean(e)}'
        to_append += f' {np.var(e)}'
    file = open(f'{songname}.csv', 'a', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(to_append.split())

# 여러 곡 테스트

In [5]:
for song in os.listdir('song'):
    data=pd.read_csv(f'{song}.csv')
    pred = model.predict(data)
    print(f"{song}은 {np.max(pred)*100}%의 확률로 '{genre_dict[np.argmax(pred)]}'입니다.")

1/1 [==============================] - 1s 770ms/step
10. 18. 14 - E-sens.wav은 99.39636588096619%의 확률로 '랩/힙합'입니다.
1/1 [==============================] - 0s 22ms/step
are-you-sleeping-classical-a-123605.wav은 100.0%의 확률로 '클래식'입니다.
1/1 [==============================] - 0s 17ms/step
Bleed It Out - Linkin Park.wav은 99.9906063079834%의 확률로 '랩/힙합'입니다.
1/1 [==============================] - 0s 17ms/step
Centuries - Fall Out Boy.wav은 42.71766543388367%의 확률로 '록/메탈'입니다.
1/1 [==============================] - 0s 17ms/step
Faint - Linkin Park.wav은 98.43941330909729%의 확률로 '랩/힙합'입니다.
1/1 [==============================] - 0s 17ms/step
Fuel - Metallica.wav은 96.7070996761322%의 확률로 '클래식'입니다.
1/1 [==============================] - 0s 17ms/step
Funny Rock.wav은 87.30812072753906%의 확률로 '클래식'입니다.
1/1 [==============================] - 0s 17ms/step
Hello.wav은 98.7089455127716%의 확률로 '클래식'입니다.
1/1 [==============================] - 0s 17ms/step
L4L(Feat. 노윤하) - Rlervine.wav은 75.69407224655151%의 확률로 '랩/힙합'입니다.
1/

# +음악 추천

In [11]:
for song in os.listdir('song'):
    data=pd.read_csv(f'{song}.csv')
    pred = model.predict(data)
    print(f"{song}은 {np.max(pred)*100}%의 확률로 '{genre_dict[np.argmax(pred)]}'입니다.")
data['song_name']=song
temp=song_data[song_data['gnr_code']==str(genre_dict[np.argmax(pred)])]
temp_mel=temp.drop(['issue_date','artist_name_basket','id','gnr_code'],axis=1)
reco_mel=pd.concat([temp_mel,data])
reco_mel_scaled=scaler.fit_transform(np.array(reco_mel.iloc[:, 1:], dtype = float))
final=pd.DataFrame(reco_mel_scaled,columns=reco_mel.columns[1:])
similarity = cosine_similarity(final)
sim_df = pd.DataFrame(similarity, index=reco_mel['song_name'], columns=reco_mel['song_name'])
series = sim_df[song].sort_values(ascending=False)
series = series.drop(song)
series = series.head().to_frame()
print(f'{song}과 유사한 곡 5개를 찾아봤어요, 한번 들어보세요~')
series = pd.merge(song_data,series,how='inner',on='song_name')
series = series.sort_values(by=[song],ascending=False)
series[[song,'song_name','artist_name_basket']]


1/1 [==============================] - 0s 60ms/step
10. 18. 14 - E-sens.wav은 99.39636588096619%의 확률로 '랩/힙합'입니다.
1/1 [==============================] - 0s 17ms/step
are-you-sleeping-classical-a-123605.wav은 100.0%의 확률로 '클래식'입니다.
1/1 [==============================] - 0s 17ms/step
Bleed It Out - Linkin Park.wav은 99.9906063079834%의 확률로 '랩/힙합'입니다.
1/1 [==============================] - 0s 18ms/step
Centuries - Fall Out Boy.wav은 42.71766543388367%의 확률로 '록/메탈'입니다.
1/1 [==============================] - 0s 18ms/step
Faint - Linkin Park.wav은 98.43941330909729%의 확률로 '랩/힙합'입니다.
1/1 [==============================] - 0s 17ms/step
Fuel - Metallica.wav은 96.7070996761322%의 확률로 '클래식'입니다.
1/1 [==============================] - 0s 17ms/step
Funny Rock.wav은 87.30812072753906%의 확률로 '클래식'입니다.
1/1 [==============================] - 0s 17ms/step
Hello.wav은 98.7089455127716%의 확률로 '클래식'입니다.
1/1 [==============================] - 0s 16ms/step
L4L(Feat. 노윤하) - Rlervine.wav은 75.69407224655151%의 확률로 '랩/힙합'입니다.
1/1

,투올더언에듀케이티드키즈 투 - Olltii.wav,song_name,artist_name_basket
1,0.963168,Non-Musical Silence/Illest Motherfucker Alive,"['JAY-Z', 'Kanye West']"
3,0.960470,Skit 1_Red,['블랭타임']
2,0.953995,(Intro) I`m so Grateful (Feat. Sizzla),['DJ Khaled']
0,0.953403,Bo! Bo! Bo!,['Boogie Down Productions']
4,0.949679,I`m Frum Dago (Feat. Lil 8),['Lunch Meat']
